In [3]:
"""
Hydrogen Fuel Cell Sensitivity Analysis Model
For use in Jupyter Notebook with interactive widgets

This model calculates key outputs for a PEM fuel cell system:
- Temperature at membrane
- Energy production
- H2O production

Author: Adapted for sensitivity analysis
Date: December 2025
"""

import numpy as np
from ipywidgets import interact, widgets

# Physical constants
FARADAY_CONSTANT = 96485  # C/mol
R_GAS_CONSTANT = 8.314    # J/(mol·K)
H2_MOLAR_MASS = 2.016e-3  # kg/mol
O2_MOLAR_MASS = 32.0e-3   # kg/mol
H2O_MOLAR_MASS = 18.015e-3 # kg/mol


def temperature_at_membrane(temp_heater,
                            distance,
                            thermal_conductivity,
                            heat_flux=5000  # W/m^2, assumed
                            ):
    """
    Calculate membrane temperature using Fourier's law of heat conduction
    
    Arguments:
        temp_heater - temperature at the heater (K)
        distance - distance from heater to membrane (m)
        thermal_conductivity - thermal conductivity of wall material (W/(m·K))
        heat_flux - heat flux through the wall (W/m^2)
    
    Returns:
        Temperature at membrane (K)
    """
    # Fourier's law: Q = k * A * (T_hot - T_cold) / d
    # Rearranging: T_membrane = T_heater - (Q/A) * d / k
    temp_drop = (heat_flux * distance) / thermal_conductivity
    temp_membrane = temp_heater - temp_drop
    
    return np.maximum(273.15, temp_membrane)  # Ensure above freezing


def nernst_voltage(temp_membrane,
                   pressure,
                   h2_utilization=0.85,
                   o2_utilization=0.50
                   ):
    """
    Calculate theoretical cell voltage using Nernst equation
    
    Arguments:
        temp_membrane - membrane temperature (K)
        pressure - operating pressure (atm)
        h2_utilization - fraction of H2 consumed (0-1)
        o2_utilization - fraction of O2 consumed (0-1)
    
    Returns:
        Cell voltage (V)
    """
    # Standard potential at 25°C
    E0 = 1.229  # V
    
    # Partial pressures (simplified: assuming remaining reactants)
    p_h2 = pressure * (1 - h2_utilization)
    p_o2 = pressure * (1 - o2_utilization)
    p_h2o = pressure * 0.5  # Assuming some water vapor
    
    # Nernst equation: E = E0 + (RT/nF) * ln(p_H2 * p_O2^0.5 / p_H2O)
    nernst_term = (R_GAS_CONSTANT * temp_membrane) / (2 * FARADAY_CONSTANT)
    
    # Avoid log of zero/negative
    if p_h2 > 0 and p_o2 > 0 and p_h2o > 0:
        voltage = E0 + nernst_term * np.log((p_h2 * np.sqrt(p_o2)) / p_h2o)
    else:
        voltage = E0
    
    return voltage


def current_density_from_membrane(membrane_thickness,
                                 temp_membrane,
                                 pressure,
                                 base_conductivity=10.0  # S/m
                                 ):
    """
    Calculate current density based on membrane properties
    
    Arguments:
        membrane_thickness - thickness of membrane (m)
        temp_membrane - membrane temperature (K)
        pressure - operating pressure (atm)
        base_conductivity - proton conductivity at reference conditions (S/m)
    
    Returns:
        Current density (A/m^2)
    """
    # Membrane resistance increases with thickness
    # Current density affected by temperature (Arrhenius) and pressure
    
    # Temperature effect (exponential relationship)
    temp_factor = np.exp((temp_membrane - 353.15) / 50)  # 353.15K = 80°C reference
    
    # Pressure effect (higher pressure improves reactant availability)
    pressure_factor = np.sqrt(pressure)
    
    # Thickness effect (inverse relationship)
    conductivity = base_conductivity * temp_factor * pressure_factor
    resistance_area = membrane_thickness / conductivity
    
    # Simplified: current density inversely proportional to resistance
    current_density = 1000 / (resistance_area + 0.0001)  # A/m^2
    
    return np.maximum(0, np.minimum(current_density, 20000))  # Cap at 20 kA/m^2


def h2_consumption_rate(h2_flowrate,
                       current_density,
                       cell_area=0.01,  # m^2
                       utilization=0.85
                       ):
    """
    Calculate actual H2 consumption based on electrochemical demand
    
    Arguments:
        h2_flowrate - H2 input flowrate (kg/s)
        current_density - current density (A/m^2)
        cell_area - active cell area (m^2)
        utilization - fraction of H2 actually consumed
    
    Returns:
        H2 consumption rate (kg/s)
    """
    # Faraday's law: n = I*t / (n_e * F)
    # For H2: 2H2 → 4H+ + 4e-, so 2 electrons per H2 molecule
    
    total_current = current_density * cell_area  # A
    
    # Molar consumption rate from current
    h2_molar_rate_from_current = (total_current / (2 * FARADAY_CONSTANT)) * utilization  # mol/s
    h2_consumption_from_current = h2_molar_rate_from_current * H2_MOLAR_MASS  # kg/s
    
    # Actual consumption is minimum of available and demanded
    actual_consumption = np.minimum(h2_flowrate * utilization, h2_consumption_from_current)
    
    return actual_consumption


def compute_fuel_cell_outputs(
        temp_heater=400,  # K (127°C)
        distance_heater_to_membrane=0.01,  # m (1 cm)
        thermal_conductivity=1.5,  # W/(m·K) - typical for ceramics
        h2_flowrate=1e-5,  # kg/s (0.01 g/s)
        o2_flowrate=8e-5,  # kg/s (0.08 g/s) - stoichiometric excess
        membrane_thickness=0.0002,  # m (200 μm)
        pressure=3.0,  # atm
        cell_area=0.01,  # m^2 (100 cm^2) - assumed
        operating_hours_per_year=8000  # hours - assumed
        ):
    """
    Main function to compute fuel cell performance metrics
    
    Returns:
        tuple: (temp_membrane, energy_production_annual, h2o_production_annual, power_output)
    """
    
    # 1. Calculate membrane temperature
    temp_membrane = temperature_at_membrane(temp_heater,
                                           distance_heater_to_membrane,
                                           thermal_conductivity)
    
    # 2. Calculate cell voltage
    voltage = nernst_voltage(temp_membrane, pressure)
    
    # 3. Calculate current density
    current_density = current_density_from_membrane(membrane_thickness,
                                                   temp_membrane,
                                                   pressure)
    
    # 4. Calculate actual H2 consumption
    h2_consumed = h2_consumption_rate(h2_flowrate,
                                     current_density,
                                     cell_area)
    
    # 5. Calculate power output
    total_current = current_density * cell_area  # A
    
    # Voltage losses (activation, ohmic, concentration)
    voltage_loss = 0.3  # V - simplified combined losses
    actual_voltage = voltage - voltage_loss
    actual_voltage = np.maximum(0.4, actual_voltage)  # Minimum voltage
    
    power_output = total_current * actual_voltage  # W
    
    # 6. Calculate annual energy production
    energy_production_annual = power_output * operating_hours_per_year / 1000  # kWh/year
    
    # 7. Calculate H2O production
    # Stoichiometry: 2H2 + O2 → 2H2O
    # 1 mol H2 produces 1 mol H2O
    h2_molar_consumed = h2_consumed / H2_MOLAR_MASS  # mol/s
    h2o_molar_produced = h2_molar_consumed  # mol/s (1:1 ratio)
    h2o_mass_rate = h2o_molar_produced * H2O_MOLAR_MASS  # kg/s
    
    # Annual H2O production
    seconds_per_year = operating_hours_per_year * 3600
    h2o_production_annual = h2o_mass_rate * seconds_per_year  # kg/year
    
    return temp_membrane, energy_production_annual, h2o_production_annual, power_output


# ============================================================================
# INTERACTIVE WIDGET FOR JUPYTER NOTEBOOK
# ============================================================================

@interact(temp_heater=widgets.FloatSlider(value=400, min=350, max=500, step=10, 
                                          description='Heater Temp (K)'),
          distance=widgets.FloatSlider(value=0.01, min=0.005, max=0.05, step=0.005,
                                      description='Distance (m)'),
          thermal_conductivity=widgets.FloatSlider(value=1.5, min=0.5, max=5.0, step=0.5,
                                                   description='Thermal Cond (W/m·K)'),
          h2_flowrate=widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1,
                                         description='H2 Flow (×10⁻⁵ kg/s)'),
          o2_flowrate=widgets.FloatSlider(value=8.0, min=1.0, max=20.0, step=1.0,
                                         description='O2 Flow (×10⁻⁵ kg/s)'),
          membrane_thickness=widgets.FloatSlider(value=200, min=50, max=500, step=50,
                                                description='Membrane (μm)'),
          pressure=widgets.FloatSlider(value=3.0, min=1.0, max=10.0, step=0.5,
                                      description='Pressure (atm)'))
def plot_fuel_cell_performance(temp_heater, distance, thermal_conductivity, 
                               h2_flowrate, o2_flowrate, membrane_thickness, pressure):
    """
    Interactive function to display fuel cell performance
    """
    # Convert units for function call
    h2_flow_actual = h2_flowrate * 1e-5  # Convert from ×10⁻⁵ kg/s to kg/s
    o2_flow_actual = o2_flowrate * 1e-5  # Convert from ×10⁻⁵ kg/s to kg/s
    membrane_thick_actual = membrane_thickness * 1e-6  # Convert from μm to m
    
    # Compute outputs
    temp_mem, energy, h2o, power = compute_fuel_cell_outputs(
        temp_heater=temp_heater,
        distance_heater_to_membrane=distance,
        thermal_conductivity=thermal_conductivity,
        h2_flowrate=h2_flow_actual,
        o2_flowrate=o2_flow_actual,
        membrane_thickness=membrane_thick_actual,
        pressure=pressure
    )
    
    # Display results
    print("=" * 60)
    print("FUEL CELL PERFORMANCE OUTPUTS")
    print("=" * 60)
    print(f"Temperature at membrane: {temp_mem:.2f} K ({temp_mem-273.15:.1f} °C)")
    print(f"Power output:            {power:.2f} W")
    print(f"Energy production:       {energy:.2f} kWh/year")
    print(f"H2O production:          {h2o:.2f} kg/year ({h2o*1000:.1f} g/year)")
    print("=" * 60)


# ============================================================================
# MONTE CARLO SIMULATION
# ============================================================================

def monte_carlo_analysis(number_sims=1000):
    """
    Perform Monte Carlo simulation by sampling random parameter values
    and analyzing their effect on outputs
    """
    import matplotlib.pyplot as plt
    
    print(f"Running Monte Carlo simulation with {number_sims} samples...")
    
    # Generate random samples for each parameter within reasonable ranges
    mc_temp_heater = np.random.uniform(350, 500, number_sims)
    mc_distance = np.random.uniform(0.005, 0.05, number_sims)
    mc_thermal_conductivity = np.random.uniform(0.5, 5.0, number_sims)
    mc_h2_flowrate = np.random.uniform(0.1e-5, 5.0e-5, number_sims)
    mc_o2_flowrate = np.random.uniform(1.0e-5, 20.0e-5, number_sims)
    mc_membrane_thickness = np.random.uniform(50e-6, 500e-6, number_sims)
    mc_pressure = np.random.uniform(1.0, 10.0, number_sims)
    
    # Store data in array for easy access
    data = np.array((mc_temp_heater,
                     mc_distance,
                     mc_thermal_conductivity,
                     mc_h2_flowrate,
                     mc_o2_flowrate,
                     mc_membrane_thickness,
                     mc_pressure))
    
    # Run simulation for all parameter combinations
    y_temp = []
    y_energy = []
    y_h2o = []
    y_power = []
    
    for i in range(number_sims):
        temp_mem, energy, h2o, power = compute_fuel_cell_outputs(
            temp_heater=data[0, i],
            distance_heater_to_membrane=data[1, i],
            thermal_conductivity=data[2, i],
            h2_flowrate=data[3, i],
            o2_flowrate=data[4, i],
            membrane_thickness=data[5, i],
            pressure=data[6, i]
        )
        y_temp.append(temp_mem)
        y_energy.append(energy)
        y_h2o.append(h2o)
        y_power.append(power)
    
    # Convert to numpy arrays
    y_temp = np.array(y_temp)
    y_energy = np.array(y_energy)
    y_h2o = np.array(y_h2o)
    y_power = np.array(y_power)
    
    # Create scatter plots - Temperature at Membrane
    fig1 = plt.figure(figsize=(16, 10))
    fig1.suptitle('Monte Carlo Analysis: Temperature at Membrane', fontsize=14, fontweight='bold')
    
    plt.subplot(241)
    plt.scatter(mc_temp_heater, y_temp, alpha=0.2, color='steelblue')
    plt.title("Heater Temperature (K)")
    plt.ylabel("Membrane Temp (K)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(242)
    plt.scatter(mc_distance, y_temp, alpha=0.2, color='steelblue')
    plt.title("Distance (m)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(243)
    plt.scatter(mc_thermal_conductivity, y_temp, alpha=0.2, color='steelblue')
    plt.title("Thermal Conductivity (W/m·K)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(244)
    plt.scatter(mc_h2_flowrate, y_temp, alpha=0.2, color='steelblue')
    plt.title("H₂ Flowrate (kg/s)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(245)
    plt.scatter(mc_o2_flowrate, y_temp, alpha=0.2, color='steelblue')
    plt.title("O₂ Flowrate (kg/s)")
    plt.ylabel("Membrane Temp (K)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(246)
    plt.scatter(mc_membrane_thickness, y_temp, alpha=0.2, color='steelblue')
    plt.title("Membrane Thickness (m)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(247)
    plt.scatter(mc_pressure, y_temp, alpha=0.2, color='steelblue')
    plt.title("Pressure (atm)")
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('mc_temperature.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Create scatter plots - Energy Production
    fig2 = plt.figure(figsize=(16, 10))
    fig2.suptitle('Monte Carlo Analysis: Energy Production', fontsize=14, fontweight='bold')
    
    plt.subplot(241)
    plt.scatter(mc_temp_heater, y_energy, alpha=0.2, color='darkgreen')
    plt.title("Heater Temperature (K)")
    plt.ylabel("Energy (kWh/year)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(242)
    plt.scatter(mc_distance, y_energy, alpha=0.2, color='darkgreen')
    plt.title("Distance (m)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(243)
    plt.scatter(mc_thermal_conductivity, y_energy, alpha=0.2, color='darkgreen')
    plt.title("Thermal Conductivity (W/m·K)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(244)
    plt.scatter(mc_h2_flowrate, y_energy, alpha=0.2, color='darkgreen')
    plt.title("H₂ Flowrate (kg/s)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(245)
    plt.scatter(mc_o2_flowrate, y_energy, alpha=0.2, color='darkgreen')
    plt.title("O₂ Flowrate (kg/s)")
    plt.ylabel("Energy (kWh/year)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(246)
    plt.scatter(mc_membrane_thickness, y_energy, alpha=0.2, color='darkgreen')
    plt.title("Membrane Thickness (m)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(247)
    plt.scatter(mc_pressure, y_energy, alpha=0.2, color='darkgreen')
    plt.title("Pressure (atm)")
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('mc_energy.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Create scatter plots - H2O Production
    fig3 = plt.figure(figsize=(16, 10))
    fig3.suptitle('Monte Carlo Analysis: H₂O Production', fontsize=14, fontweight='bold')
    
    plt.subplot(241)
    plt.scatter(mc_temp_heater, y_h2o, alpha=0.2, color='darkred')
    plt.title("Heater Temperature (K)")
    plt.ylabel("H₂O Production (kg/year)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(242)
    plt.scatter(mc_distance, y_h2o, alpha=0.2, color='darkred')
    plt.title("Distance (m)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(243)
    plt.scatter(mc_thermal_conductivity, y_h2o, alpha=0.2, color='darkred')
    plt.title("Thermal Conductivity (W/m·K)")
    plt.grid(True, alpha=0.3)
    
    plt.subplot(244)
    plt.scatter(mc_h2_flowrate, y_h2o, alpha=0.2, color='darkred')
    plt.title("H₂ Flowrate (kg/s)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(245)
    plt.scatter(mc_o2_flowrate, y_h2o, alpha=0.2, color='darkred')
    plt.title("O₂ Flowrate (kg/s)")
    plt.ylabel("H₂O Production (kg/year)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(246)
    plt.scatter(mc_membrane_thickness, y_h2o, alpha=0.2, color='darkred')
    plt.title("Membrane Thickness (m)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(247)
    plt.scatter(mc_pressure, y_h2o, alpha=0.2, color='darkred')
    plt.title("Pressure (atm)")
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('mc_h2o.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*80)
    print("MONTE CARLO SIMULATION RESULTS")
    print("="*80)
    print(f"\nMembrane Temperature:")
    print(f"  Mean:   {np.mean(y_temp):.2f} K ({np.mean(y_temp)-273.15:.2f} °C)")
    print(f"  Std:    {np.std(y_temp):.2f} K")
    print(f"  Min:    {np.min(y_temp):.2f} K ({np.min(y_temp)-273.15:.2f} °C)")
    print(f"  Max:    {np.max(y_temp):.2f} K ({np.max(y_temp)-273.15:.2f} °C)")
    
    print(f"\nEnergy Production:")
    print(f"  Mean:   {np.mean(y_energy):.2f} kWh/year")
    print(f"  Std:    {np.std(y_energy):.2f} kWh/year")
    print(f"  Min:    {np.min(y_energy):.2f} kWh/year")
    print(f"  Max:    {np.max(y_energy):.2f} kWh/year")
    
    print(f"\nH₂O Production:")
    print(f"  Mean:   {np.mean(y_h2o):.2f} kg/year")
    print(f"  Std:    {np.std(y_h2o):.2f} kg/year")
    print(f"  Min:    {np.min(y_h2o):.2f} kg/year")
    print(f"  Max:    {np.max(y_h2o):.2f} kg/year")
    
    print(f"\nPower Output:")
    print(f"  Mean:   {np.mean(y_power):.2f} W")
    print(f"  Std:    {np.std(y_power):.2f} W")
    print(f"  Min:    {np.min(y_power):.2f} W")
    print(f"  Max:    {np.max(y_power):.2f} W")
    print("="*80)
    
    return data, y_temp, y_energy, y_h2o, y_power


# ============================================================================
# SCATTER PLOTS FOR SENSITIVITY ANALYSIS
# ============================================================================

def sensitivity_analysis_plots():
    """
    Generate scatter plots for one-at-a-time sensitivity analysis
    Shows how each input parameter affects the three outputs
    """
    import matplotlib.pyplot as plt
    
    # Baseline parameters
    baseline = {
        'temp_heater': 400,
        'distance': 0.01,
        'thermal_conductivity': 1.5,
        'h2_flowrate': 1e-5,
        'o2_flowrate': 8e-5,
        'membrane_thickness': 0.0002,
        'pressure': 3.0
    }
    
    # Define parameter ranges for sensitivity analysis
    param_ranges = {
        'temp_heater': np.linspace(350, 500, 20),
        'distance': np.linspace(0.005, 0.05, 20),
        'thermal_conductivity': np.linspace(0.5, 5.0, 20),
        'h2_flowrate': np.linspace(0.1e-5, 5.0e-5, 20),
        'o2_flowrate': np.linspace(1.0e-5, 20.0e-5, 20),
        'membrane_thickness': np.linspace(50e-6, 500e-6, 20),
        'pressure': np.linspace(1.0, 10.0, 20)
    }
    
    # Labels for plotting
    param_labels = {
        'temp_heater': 'Heater Temperature (K)',
        'distance': 'Distance (m)',
        'thermal_conductivity': 'Thermal Conductivity (W/m·K)',
        'h2_flowrate': 'H₂ Flowrate (kg/s)',
        'o2_flowrate': 'O₂ Flowrate (kg/s)',
        'membrane_thickness': 'Membrane Thickness (m)',
        'pressure': 'Pressure (atm)'
    }
    
    # Storage for results
    results = {param: {'temp_mem': [], 'energy': [], 'h2o': [], 'power': []} 
               for param in param_ranges.keys()}
    
    # Run sensitivity analysis for each parameter
    print("Running sensitivity analysis...")
    for param_name, param_values in param_ranges.items():
        for value in param_values:
            # Create parameter dict with baseline values
            params = baseline.copy()
            params[param_name] = value  # Vary only this parameter
            
            # Compute outputs
            temp_mem, energy, h2o, power = compute_fuel_cell_outputs(**params)
            
            # Store results
            results[param_name]['temp_mem'].append(temp_mem)
            results[param_name]['energy'].append(energy)
            results[param_name]['h2o'].append(h2o)
            results[param_name]['power'].append(power)
    
    # Create plots
    fig, axes = plt.subplots(3, 7, figsize=(24, 12))
    fig.suptitle('Fuel Cell Sensitivity Analysis: One-at-a-Time Approach', 
                 fontsize=16, fontweight='bold')
    
    output_names = ['temp_mem', 'energy', 'h2o']
    output_labels = ['Temperature at Membrane (K)', 
                     'Energy Production (kWh/year)', 
                     'H₂O Production (kg/year)']
    
    # Plot each output vs each parameter
    for i, (output_name, output_label) in enumerate(zip(output_names, output_labels)):
        for j, param_name in enumerate(param_ranges.keys()):
            ax = axes[i, j]
            
            # Get x and y data
            x_data = param_ranges[param_name]
            y_data = results[param_name][output_name]
            
            # Create scatter plot
            ax.scatter(x_data, y_data, alpha=0.6, s=40, color='steelblue')
            
            # Add title and labels
            if i == 0:
                ax.set_title(param_labels[param_name], fontsize=10, fontweight='bold')
            if j == 0:
                ax.set_ylabel(output_label, fontsize=9)
            
            # Format x-axis for scientific notation if needed
            if param_name in ['h2_flowrate', 'o2_flowrate', 'membrane_thickness']:
                ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
            
            ax.grid(True, alpha=0.3)
            ax.tick_params(labelsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*80)
    print("SENSITIVITY ANALYSIS SUMMARY")
    print("="*80)
    
    for param_name in param_ranges.keys():
        print(f"\n{param_labels[param_name]}:")
        print(f"  Temperature range: {min(results[param_name]['temp_mem']):.1f} - {max(results[param_name]['temp_mem']):.1f} K")
        print(f"  Energy range:      {min(results[param_name]['energy']):.1f} - {max(results[param_name]['energy']):.1f} kWh/year")
        print(f"  H₂O range:         {min(results[param_name]['h2o']):.1f} - {max(results[param_name]['h2o']):.1f} kg/year")


if __name__ == "__main__":
    # Example calculation with baseline parameters
    print("Run this code in a Jupyter notebook to see the interactive widgets!\n")
    print("To generate sensitivity analysis plots, run: sensitivity_analysis_plots()\n")
    
    temp_mem, energy, h2o, power = compute_fuel_cell_outputs()
    
    print("Baseline Fuel Cell Performance:")
    print(f"Temperature at membrane: {temp_mem:.2f} K ({temp_mem-273.15:.2f} °C)")
    print(f"Power output: {power:.2f} W")
    print(f"Annual energy production: {energy:.2f} kWh/year")
    print(f"Annual H2O production: {h2o:.2f} kg/year ({h2o*1000:.1f} g/year)")

interactive(children=(FloatSlider(value=400.0, description='Heater Temp (K)', max=500.0, min=350.0, step=10.0)…

Run this code in a Jupyter notebook to see the interactive widgets!

To generate sensitivity analysis plots, run: sensitivity_analysis_plots()

Baseline Fuel Cell Performance:
Temperature at membrane: 366.67 K (93.52 °C)
Power output: 182.64 W
Annual energy production: 1461.09 kWh/year
Annual H2O production: 457.07 kg/year (457073.3 g/year)
